In [21]:
import numpy as np
from utils import *
import cmath
import math

In [22]:
def make_const_code(m):
    const = []
    code = []
    quad = int(m/4)
    sq = math.sqrt(m)
    N = int(math.sqrt(quad))
    for k in range(4):
        for i in range(N):
            for j in range(N):
                s1 = -1 if k < 2 else 1
                s2 = -1 if k % 2 == 0 else 1 
                re = s1*(2*j+1)*(3/2)*(1/(sq-1))
                im = s2*(2*i+1)*(3/2)*(1/(sq-1))
                bits = bin(quad*k + N*i + j)[2:]
                byte = '00000000'[len(bits):] + bits
                const.append(complex(re, im))
                code.append(byte)

    return const, code

In [23]:
K = 10
const, code = make_const_code(256)

In [53]:
def to_qam(s):
    s = 'aaaaaaaaaa' + s
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        byte = '00000000'[len(bits):] + bits
        result.append(const[int("".join(str(k) for k in byte), 2)])
    return np.array(result)

In [59]:
## Transmitter
initial_str = read_file("initial.txt")
result = to_qam(initial_str[0])
serialize_complex(result, "input.txt")

In [60]:
## Client
! python3 client.py --input_file=input.txt --output_file=output.txt

In [61]:
def from_qam(s):
    phase = s[0:9]
    s = s[10:len(s)]
    tmp = ''
    theta = cmath.phase(sum(phase)/math.sqrt(10))
    for c in s:
        shifted = abs(c)*np.exp(1j*(theta+np.angle(c)))
        idx = np.argmin([abs(shifted-e) for e in const])
        tmp += code[idx]
    return [tmp[i:i+8] for i in range(0, len(tmp), 8)]

In [62]:
## Theta estimation receiver
content = read_file("output.txt")
content = [complex(c) for c in content]

comps = from_qam(content)
finalText = comps_to_string(comps)

write_final(finalText)
compute_score()

Result is :  ãxùùxxyòøðãxxâóxôyùãxùøxxyóøðãyxâòpôyùÛxùùyxyòùðãyxâòxôyøãxøùyxyóùðãyxâòpôyøãx
Number of different characters :  75
